# Setting Up The SQLite Server
Below is the code to import the sqlite3 library that is included with the Python standard library package.

In [1]:
import sqlite3

Next is the code to create/connect with the database

In [3]:
connection = sqlite3.connect('movies.db')

Now the next code will create a cursor object so we can execute SQL statements

In [4]:
cursor = connection.cursor()

## Drop Tables
Drop all the tables from the database to start fresh.

In [5]:
cursor.execute('DROP TABLE IF EXISTS movies_box_office')

## Create Tables 
### Main Table From imdb dataset (movies_box_office)

In [6]:
cursor.execute('''CREATE TABLE IF NOT EXISTS movies_box_office
               (
                    movies_box_office INTEGER PRIMARY KEY NOT NULL,
                    movie_title VARCHAR(100) NOT NULL,
                    world_wide_bo INTEGER,
                    domestic_amount INTEGER,
                    domestic_percentage FLOAT,
                    foreign_amount INTEGER,
                    foreign_percentage FLOAT
                )''')

In [7]:
import pandas as pd

# List of file names
filenames = ['./datasets/box-office/ranking_summary_2018.csv', './datasets/box-office/ranking_summary_2019.csv', './datasets/box-office/ranking_summary_2020.csv', './datasets/box-office/ranking_summary_2021.csv', './datasets/box-office/ranking_summary_2022.csv']

# Read each CSV file into a DataFrame and store them in a list
dataframes = [pd.read_csv(f) for f in filenames]

# Concatenate all the dataframes into one
combined_dataframe = pd.concat(dataframes, ignore_index=True)

In [8]:
combined_dataframe

,rank,title,worldwide,domestic,domestic_pct,foreign,foreign_pct
0,1,Avengers: Infinity War,"$2,048,359,754","$678,815,482",33.1%,"$1,369,544,272",66.9%
1,2,Black Panther,"$1,346,913,161","$700,059,566",52%,"$646,853,595",48%
2,3,Jurassic World: Fallen Kingdom,"$1,308,467,944","$417,719,760",31.9%,"$890,748,184",68.1%
3,4,Incredibles 2,"$1,242,805,359","$608,581,744",49%,"$634,223,615",51%
4,5,Aquaman,"$1,151,961,807","$335,061,807",29.1%,"$816,900,000",70.9%
...,...,...,...,...,...,...,...
995,196,Lesson in Murder,"$7,348,964",-,-,"$7,348,964",100%
996,197,Family Affairs,"$7,330,755",-,-,"$7,330,755",100%
997,198,Listy do M. 5,"$7,328,061",-,-,"$7,328,061",100%
998,199,Laid-Back Camp Movie,"$7,317,913",-,-,"$7,317,913",100%
